In [1]:
import basic_image_operations as basic_ops
import geometrical_image_operations as geo_ops
import traffic_sign_detection as tsd
import colors
import numpy as np

stop_img = basic_ops.load_image(image_path='traffic_sign_templates/stop_signs/stop.jpg')
vfa_img = basic_ops.load_image(image_path='traffic_sign_templates/vfa_signs/vfa.jpg')
vfs_img = basic_ops.load_image(image_path='traffic_sign_templates/vfs_signs/vfs.jpg')
templates = [stop_img, vfa_img, vfs_img]
color_functions = [colors.is_strong_red, colors.is_strong_red, colors.is_strong_yellow]
templates = [geo_ops.resize_image(image=template, target_height=200, target_width=200) for template in templates]

Image loaded from traffic_sign_templates/stop_signs/stop.jpg
Image loaded from traffic_sign_templates/vfa_signs/vfa.jpg
Image loaded from traffic_sign_templates/vfs_signs/vfs.jpg


In [3]:
for template, color_function in zip(templates, color_functions):
    img = np.copy(template)
    for y in range(template.shape[0]):
        for x in range(template.shape[1]):
            hsv = colors.bgr_to_hsv(template[y, x])
            h, s, v = hsv[0], hsv[1], hsv[2]
            if color_function(h, s, v):
                continue
            img[y, x] = 0
    mask = colors.get_mask(image=img, color_function=color_function)
    blobs = tsd.get_blobs(mask=mask)
    boxed_image, center_positions = tsd.draw_bounding_boxes(image=template, blobs=blobs, min_box_area=5000)
    print(center_positions)
    basic_ops.show_image(image=boxed_image, title='boxed_image')



[(100, 100, 32041)]
Image displayed
[(100, 100, 37248)]
Image displayed
[(100, 100, 12321)]
Image displayed
